# Лабораторная работа № 8

Генерация текста на основе “Алисы в стране чудес”
    
Задачи:

   1. Ознакомиться с генерацией текста
   2. Ознакомиться с системой Callback в Keras

# Цель работы:
Рекуррентные нейронные сети также могут быть использованы в качестве генеративных
моделей.
Это означает, что в дополнение к тому, что они используются для прогнозных моделей
(создания прогнозов), они могут изучать последовательности проблемы, а затем
генерировать совершенно новые вероятные последовательности для проблемной
области.
Подобные генеративные модели полезны не только для изучения того, насколько хорошо
модель выявила проблему, но и для того, чтобы узнать больше о самой проблемной
области.

In [1]:
import numpy
import sys
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [3]:
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  144522
Total Vocab:  48


In [4]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  144422


In [5]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy',
optimizer='adam')

In [ ]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, y, epochs=50, batch_size=128,
callbacks=callbacks_list)

Epoch 1/50
1129/1129 [==============================] - ETA: 0s - loss: 2.9630
Epoch 1: loss improved from inf to 2.96296, saving model to weights-improvement-01-2.9630.hdf5
1129/1129 [==============================] - 1076s 930ms/step - loss: 2.9630
Epoch 2/50
 515/1129 [============>.................] - ETA: 8:42 - loss: 2.7942

In [ ]:
# load the network weights
filename = "weights-improvement-50-1.6400.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy',
optimizer='adam')
int_to_char = dict((i, c) for i, c in enumerate(chars))

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):

    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print ("\nDone.")

In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):

    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print ("\nDone.")

In [ ]:
import numpy
import sys
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy',
optimizer='adam')

# define the checkpoint
tb_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32,
                                             write_graph=True, write_grads=False, write_images=False,
                                             embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None,
                                             embeddings_data=None, update_freq='epoch')

model.fit(X, y, epochs=10, batch_size=512, callbacks=[tb_callback])

In [ ]:
# load the network weights
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy',
optimizer='adam')


class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
            if (epoch + 1)%1 == 0:
                # pick a random seed
                start = numpy.random.randint(0, len(dataX)-1)
                pattern = dataX[start]
                print ("Seed:")
                print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
    
            # generate characters
                for i in range(1000):
                    x = numpy.reshape(pattern, (1, len(pattern), 1))
                    x = x / float(n_vocab)
                    prediction = model.predict(x, verbose=0) 
                    index = numpy.argmax(prediction)
                    result = int_to_char[index]
                    seq_in = [int_to_char[value] for value in pattern]
                    sys.stdout.write(result)
                    pattern.append(index)
                    pattern = pattern[1:len(pattern)]
                    
                print ("\nDone.")
                
# define the checkpoint
#tb_callback = keras.callbacks.Callback.CustomCallback (log_dir='./logs', histogram_freq=0, batch_size=32,
#write_graph=True, write_grads=False, write_images=False,
#embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None,
#embeddings_data=None, update_freq='epoch')

model.fit(X, y, epochs=20, batch_size=512, callbacks=[CustomCallback()])
